# オートエンコーダを使用した学習処理を実行するプログラム

## 実行環境のライブラリバージョンの調整

演習で使用するライブラリのバージョンをプログラムの内容に合わせて入れ替えます。

In [ ]:
# tensorflowのバージョンをラズパイの推論処理のプログラム環境に合わせて変更する
!pip install tensorflow==1.14.0
!pip install tensorboard==1.14.0
!pip install tensorflow-estimator==1.14.0
!pip install scikit-learn==0.21.2
!pip install Keras==2.2.4
!pip install Keras-Applications==1.0.8
!pip install Keras-Preprocessing==1.1.0



## 必要なモジュールのインポート
処理に必要な各種のライブラリモジュールをインポートします。

In [ ]:
# 必要なライブラリのインポート
import tensorflow as tf
#import tensorflow.contrib.eager as tfe
from tensorflow.keras.models import Sequential, Model, save_model
from tensorflow.keras.layers import Activation, Dense, Input
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
#import os
import numpy as np
#import pandas as pd
import seaborn as sns
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import pickle

## Googleドライブのマウント

Google Colaboratoryで実行する場合は、Googleドライブをマウントし、ファイルの読み書きを行えるようにします。
実行すると、プログラムの下の部分に認証コードを取得するためのリンクが表示されるので、そのリンクをクリックし、必要に応じてGoogleのログイン処理やアクセス許可の処理を行います。その後、認証用のコードが表示されるので、そのコードをコピーしたら、最初に表示されていたリンクの下の入力欄に認証コードを入力してEnterキーを押して下さい。"Mounted at /content/drive"と表示されたらマウント処理は完了です。

In [ ]:
  # Colaboratory上で実行することでGoogleドライブのマウント処理を行う
  from google.colab import drive
  drive.mount('/content/drive')
  # 作業フォルダのパス
  work_path = '/content/drive/My Drive/work/'


## 学習用データの読み込み

あらかじめ用意した学習用データの読み込みを行います。

In [ ]:
filename = work_path + 'training_60s.csv'

# csvファイルから元データを読み込む
data_train = np.loadtxt(filename, delimiter=',')

# 読み込んだデータのサイズを表示
print('学習用データ: ', data_train.shape)

## データの正規化

読み込んだデータを正規化します。また、正規化で使用したパラメタ情報は推論時にも必要となるためファイルに保存します。

In [ ]:
# 値の範囲を正規化
scaler = MinMaxScaler()
data_train_std = scaler.fit_transform(data_train)

# 正規化に使用したパラメタを推論時に使用するためファイルに保存
pickle.dump(scaler, open(work_path+'scaler.pkl', 'wb'))

## 訓練用データと検証用データの分割（ホールドアウト法）

過学習かどうかを確認できるようにするために、学習用データと検証用データに分割します。全体を8:2で分割します。


In [ ]:
# 検証用データの割合を20%として、データを分割する
X_trn, X_tst = train_test_split(data_train_std, test_size=0.2)

# 分割結果のサイズを表示
print('訓練用データ X_trn: ',X_trn.shape)
print('検証用データ X_tst: ',X_tst.shape)

## AutoEncoderモデルの構成を定義

学習で使用するAutoEncoderモデルの構成を定義します。
レイヤの構成はエンコーダー部とデコーダー部をそれぞれ２層としています。

In [ ]:
# AutoEncoderのネットワーク構造を定義
model = Sequential() # 層を重ねて定義していくタイプのモデル
model.add(Dense(40, input_dim = 40, activation='relu'))
model.add(Dense(20, activation='relu'))
model.add(Dense(5, activation='relu', name = 'encoder'))
model.add(Dense(20, activation='relu'))
model.add(Dense(40, activation='sigmoid'))

# 学習モデルのコンパイル（実行準備）
model.compile(loss = 'mse', optimizer ='adam')

## 学習処理の実行

用意したAutoEncoderのモデルと学習用データを使って学習処理を実行します。実行が完了したら、学習済モデルをファイルとして保存します。

In [ ]:
# 学習処理の実行
hist = model.fit(X_trn, X_trn, epochs=100, batch_size=128, validation_data=(X_tst, X_tst))

In [ ]:
# 学習済みモデルの保存
tf.keras.experimental.export_saved_model(model, 'saved_model', serving_only=True)

In [ ]:
# 学習済みモデルを転送しやすいように圧縮
!tar -zcvf saved_model.tar.gz saved_model

In [ ]:
# 学習済みモデルをGoogleドライブにコピー
!cp saved_model.tar.gz drive/MyDrive/work/

## 学習結果の確認

学習中の誤差の推移をグラフでプロットし、収束状況や過学習の状況を確認します。

In [ ]:
# エポック経過にともなう二乗平均誤差の履歴のプロット
plt.plot(hist.history['loss'],"b",label="Training loss",)
plt.plot(hist.history['val_loss'],"r",label="Validation loss")
plt.title('Loss history')
plt.xlabel('Epochs')
plt.ylabel('Loss, [mse]')
plt.legend(loc='upper right')
plt.yscale('log')
plt.show()


以下はテスト用のコードなので本番環境では不要
---
学習時のmseの値の変化と推論時のmseの値がおおむね一致しているのかどうかを確認するためのコード


In [ ]:
#テストコード

output = model.predict(X_tst)

In [ ]:
print(X_trn)

In [ ]:
print(output)

In [ ]:
# 以下で計算したら大体あっていた　2022/3/4 11:30
from sklearn.metrics import mean_squared_error

mse = mean_squared_error(X_tst, output)

In [ ]:
mse = tf.keras.metrics.mean_squared_error(X_trn, output)

In [ ]:
print(mse)

In [ ]:
import pkg_resources
for dist in pkg_resources.working_set:
  print(dist)